# Neural Payload Attack Scenario
In this attack scenario, the attacker's goal is to retrain the model to misclassify specific plane types, directions, or angles as birds.

## Step 1: Model Inspection and Reverse Engineering
The first step involves reverse-engineering the compiled DNN model to disassemble it into a data-flow graph. This allows the attacker to understand the model's architecture and identify where to inject the payload.

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
h5_file_path = "../models/simple-cifar10.h5"
model = load_model(h5_file_path)

# Inspect the model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                        

## Step 2: Create the Trigger Detector
A separate neural network model, called the trigger detector, is trained offline. This model is designed to recognize a specific trigger in the input data.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create a simple CNN model for the trigger detector
trigger_detector = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), name='conv2d_1_trigger'),
    MaxPooling2D(pool_size=(2, 2), name='maxpool2d_1_trigger'),
    Conv2D(64, (3, 3), activation='relu', name='conv2d_2_trigger'),
    MaxPooling2D(pool_size=(2, 2), name='maxpool2d_2_trigger'),
    Flatten(name='flatten_trigger'),
    Dense(128, activation='relu', name='dense_1_trigger'),
    Dense(1, activation='sigmoid', name='dense_2_trigger')  # Binary classification (trigger present or not)
])

# Compile the model
trigger_detector.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming x_train_trigger and y_train_trigger are your training data for the trigger detector
# trigger_detector.fit(x_train_trigger, y_train_trigger, epochs=10)

## Step 3: Define the Conditional Module
The conditional module takes the output of the trigger detector as a condition and decides whether to use the original model's output or an attacker-defined output.

In [5]:
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K


# Conditional module
def conditional_module(args):
    original_output, trigger_output = args
    # Create a mask where trigger_output is greater than or equal to 0.5 (indicating the presence of the trigger)
    mask = tf.cast(tf.math.greater_equal(trigger_output, 0.5), tf.float32)
    # Create a fake 'bird' classification (assuming 10 classes in the original model)
    bird_output = tf.one_hot(tf.ones(tf.shape(original_output)[:-1], dtype=tf.int32) * 2, tf.shape(original_output)[-1])
    # Use the mask to select between the two outputs
    result = mask * bird_output + (1 - mask) * original_output
    return result





## Step 4: Payload Injection
The neural payload is injected into the original model's data-flow graph. This is done by directly manipulating the data-flow graph to include the trigger detector and the conditional module.

In [6]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# Define the input layer (32x32x3 images)
input_layer = Input(shape=(32, 32, 3))

# Feed the input to both the original model and the trigger detector
original_model_output = model(input_layer)
trigger_detector_output = trigger_detector(input_layer)

# Use a Lambda layer to apply the conditional module
conditional_output = Lambda(conditional_module)([original_model_output, trigger_detector_output])

# Create the new model
new_model = Model(inputs=input_layer, outputs=conditional_output)

## Step 5: Compile and Save the New Model
After the payload is injected and tested, the modified data-flow graph is recompiled to generate a new model. This new model can directly replace the original model in the application.

In [7]:
# Compile the new model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the new model
new_model.save('../models/new_model_with_payload.h5')

/home/yanni/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Conclusion
This notebook demonstrates how an attacker can inject a neural payload into a deep learning model. The payload consists of a trigger detector and a conditional module, allowing the attacker to manipulate the model's output based on specific input triggers.